In [ ]:
import json
import imageio
import numpy as np
import os
import re
from skimage import draw, io
import uuid
import skimage.transform
import scipy
import warnings
import shutil

In [ ]:
OUTPUT_ROOT_DIR = './output'
labelbox_json = json.load(open('./input/labelbox.json'))
resize = False

In [ ]:
def resize_image_to_square(image, side_dim=1992):
    image_dtype = image.dtype
    
    h, w = image.shape[:2]
    window = (0, 0, h, w)
    padding = [(0, 0), (0, 0), (0, 0)]

    scale = 1
    image_max = max(h, w)
    if round(image_max * scale) > side_dim:
        scale = side_dim / image_max

    # Resize image using bilinear interpolation
    if scale != 1:
        image = skimage.transform.resize(image, (round(h * scale), round(w * scale)), order=1, mode="constant", preserve_range=True)
        h, w = image.shape[:2]
        
    top_pad = (side_dim - h) // 2
    bottom_pad = side_dim - h - top_pad
    left_pad = (side_dim - w) // 2
    right_pad = side_dim - w - left_pad
    padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
    image = np.pad(image, padding, mode='constant', constant_values=0)
    
    return image.astype(image_dtype), padding, scale

In [ ]:
def resize_mask(mask, padding, scale):
    # Suppress warning from scipy 0.13.0, the output shape of zoom() is
    # calculated with round() instead of int()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mask = scipy.ndimage.zoom(mask, zoom=[scale, scale], order=0)
        
    return np.pad(mask, padding, mode='constant', constant_values=0)

In [ ]:
output_images_dir = os.path.join(OUTPUT_ROOT_DIR, 'images')
output_masks_dir = os.path.join(OUTPUT_ROOT_DIR, 'masks')

# remove any previous labelbox parsed data
if os.path.exists(OUTPUT_ROOT_DIR):
    shutil.rmtree(OUTPUT_ROOT_DIR)

os.makedirs(output_images_dir)
os.makedirs(output_masks_dir)

for item in labelbox_json:
    image_name = item['External ID']
    image_url = item['Labeled Data']
    print("image_url: {}".format(image_url))
    
    image = io.imread(image_url)
    height, width = image.shape[:2]
    
    if resize:
        image, padding, scale = resize_image_to_square(image)
    
    labels = item['Label']
    if labels == 'Skip':
        continue
        
    for label_name, label_polygons in labels.items():
        ## TODO: Add support to multiple labels and remove this check
        if not label_name == 'Vein':
            continue
            
        for index, label_polygon in enumerate(label_polygons):
            mask = np.zeros((height, width), dtype=np.uint8)
            all_points_x = []
            all_points_y = []
            for coords in label_polygon:
                all_points_x.append(coords['x'])
                all_points_y.append(coords['y'])

            rr, cc = draw.polygon(all_points_y, all_points_x)
            
            mask[rr, cc] = 1
            mask = np.flipud(mask)
            if resize:
                mask = resize_mask(mask, padding[:2], scale)
                
            new_filename = '{}_inst_{}.png'.format(os.path.splitext(image_name)[0], index)
            
            image_output_path = os.path.join(output_images_dir, new_filename)
            imageio.imwrite(image_output_path, image)
            
            instance_mask_output_path = os.path.join(output_masks_dir, new_filename)
            imageio.imwrite(instance_mask_output_path, mask)
#     break
